### Deploy and Interact with Consent based Smart Contract
The aim of this notebook is to compile, deploy and interact with consent based smart contract

### Compile contract from file

In [32]:
import json
import web3
from datetime import datetime
from web3 import Web3
from solcx import compile_source
import solcx
#from web3.contract import ConciseContract

In [33]:
# Read in Model contract code
with open('./data/Model.sol', 'r') as file:
    contract_source_code = file.read()

In [34]:
solcx.install_solc('0.6.2')
# Compile & Store Compiled source code
compiled_sol = solcx.compile_source(contract_source_code,solc_version='0.6.2')

In [35]:
#solcx.install_solc('0.6.2')

In [36]:
#compiled_sol

In [37]:
# Extract full interface as dict from compiled contract
contract_interface = compiled_sol['<stdin>:Model']

In [38]:
#contract_interface

In [39]:
# Extract abi and bytecode
abi = contract_interface['abi']
bytecode = contract_interface['bin']

### Deploy

In [40]:
# Use Ganache for web3 instance
w3 = Web3(Web3.HTTPProvider("HTTP://127.0.0.1:7545"))

In [41]:
# Use local Ganache GUI on macOS
#w3 = Web3(Web3.HTTPProvider("HTTP://192.168.72.1:7545"))

In [42]:
# Set pre-funded ganache account #0 as sender
w3.eth.defaultAccount = w3.eth.accounts[0]

In [43]:
w3.eth.accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

In [44]:
w3.eth.accounts[0]

'0x23cA5505ca6D97f439C66AaA6b371216406CDE4F'

The default `eth.defaultAccount` address is used as the default "from" property for transaction dictionaries if no other explicit "from" property is specified.

In [45]:
# Create contract blueprint
Contract = w3.eth.contract(abi=abi, bytecode=bytecode)

In [46]:
# Submit the transaction that deploys the contract
tx_hash = Contract.constructor().transact({"from": w3.eth.accounts[0]})

In [47]:
tx_hash

HexBytes('0x38b870a9b3f09f6a24d1d0e6dd8214e849e204db73a4674e6575eb8580014c24')

### Obtain Transcation Receipt

In [48]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

In [49]:
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number",contract_block,".")

The contract is deployed with block number 73 .


In [50]:
# With obtain the final address of the contract 
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

The contract has the address 0x6Da5800336B730723B2b9242c12b0ADb6467dA6A .


### Interact with contract

In [51]:
# Create python instance of deployed contract
Contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [52]:
# Extract default accounts created by ganache
accounts = w3.eth.accounts

In [53]:
accounts

['0x23cA5505ca6D97f439C66AaA6b371216406CDE4F',
 '0xe6Cf0B440a3a1Eb03dD8fE8007c8D770FF050002',
 '0x31be7A34eb0bc2e93c7764397E926d2Fbc04dAa9',
 '0xeF7e89b08d4ed534F1d693a1DAB081475F741526',
 '0x0ECeCc018538979248fbC7060262EB2C7e6FeE34',
 '0x1aDe578ad2a1dE610e4287683365c8660f8FB6fb',
 '0xC7034366279A7BA40F9f5925A5AFe5117f723300',
 '0xF3E92676BF29f956Dca7C6cb7C90dA985e3FdC4D',
 '0xe08B8414EB168b8f3EA3B118132B4eeECabb761D',
 '0x452cE57A1e9Ef53077Fa1F61e7cD0B6a4b416165']

**Certification**  
```solidity

        function addCertifier(address newCertifier) public onlyOwner {}
        function deleteCertifier(address oldCertifier) public onlyOwner {}
        function checkIfCertified(address certified) public view returns(bool) {}
```

In [54]:
#print("Gas", Contract.functions.UploadDataPrimaryCategory(test_address0, False, True, True, True, True).estimateGas())
startTime = datetime.now()
#adding an event listener so we can check the eventss
event_filter = Contract.events['publishedModel'].create_filter(fromBlock='latest')


In [55]:
#checking if adding a new account to the certification works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before adding, certified = " + str(tx_hash))
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after adding, certified = " + str(tx_hash))

before adding, certified = False
after adding, certified = True


In [56]:
#checking if removing a certifier also works, we need to set an account from which the transact comes from as this function can only be called by the owner.
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("before removing, certified = " + str(tx_hash))
tx_hash = Contract.functions.deleteCertifier(accounts[1]).transact({'from':accounts[0]})
tx_hash = Contract.functions.checkIfCertified(accounts[1]).call()
print("after removing, certified = " + str(tx_hash))

before removing, certified = True
after removing, certified = False


In [57]:
print(w3.eth.get_block('latest'))

AttributeDict({'hash': HexBytes('0x32f77031f83c3e4c9e47c9370c95bab6402b3e3b66f9364f590970df959995c4'), 'parentHash': HexBytes('0x945387bd0a3a9269e474f03a398e9d86fb2b15971b12111c63bb1da0b2a822c4'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'miner': '0x0000000000000000000000000000000000000000', 'stateRoot': HexBytes('0x3b8ab6c81b2993a73e52c74b7991fa6abe7f6f9c422ef17b97a5678e0cb2e908'), 'transactionsRoot': HexBytes('0xab2ecea8d1931bf56e3ca013796e8feb15ace5e427a592d9ee4fa2677b76925f'), 'receiptsRoot': HexBytes('0x76b5d9bdb72b9ae5bd41b7fa634c1309f7cd22ee80fbe7d797180e94b7b73843'), 'logsBloom': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [58]:
endTimeDataProvider = datetime.now()
elapsedTimeDataProvider = endTimeDataProvider - startTime
print("elapsedTimeDataProvider", elapsedTimeDataProvider)

elapsedTimeDataProvider 0:00:04.481975


In [59]:
#tx_hash

**Adding models**  
```solidity
        function publishModel(
                string memory _link,
                bytes memory _hash) public onlyOwner providerGasCost
```

**Updating Models**  
```solidity
        function updateModel(
                string memory _link,
                bytes memory newHash) public onlyOwner providerGasCost
```

**Certifying models**  
```solidity
        function certifyModel(bytes memory _hash) public certifier
```


**Getters and Setters for variables**  
```solidity
        function getVersion(bytes32 _hash)public view returns(uint256){}
        function getCertfication(uint256 _version) public view returns(bool){}
        function setLink(string memory _link) public onlyOwner(){}
        function getLink() public view returns(string memory){}
        function getModelCardHash() public view returns(bytes32){}


In [61]:
#Lets look at how the functions work whenever we are trying to upload a model,
#In here im using a test hash, this does not refer to a real hash, //you can add a real model here 

test_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777b93592d8d4d4a4096c57876a91c84f4a712')
#basically if model card hash is not 32 bytes needs to be changed
test_link = 'https/:www.somelinktomodel.com'
#please let me know if you have any troubles with this part, it might be that the model card is not a bytes32 hash, in that case i need to change the smart contract,
tx_hash = Contract.functions.publishModel(test_link, test_hash).transact({'from': accounts[0]})
#this model is not callable twice on the same contract, as it will recognise that the contract is published, keep that in mind while testing other functions in the same cell

#lets see if everything prints accordingly
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

https/:www.somelinktomodel.com, the link to the model
the model is certified: False


In [62]:
#lets see if we can print the event
def handle_event(event):
    print("Event:", event)

# Start event listener
for event in event_filter.get_all_entries():
    handle_event(event)

Event: AttributeDict({'args': AttributeDict({'sender': '0x23cA5505ca6D97f439C66AaA6b371216406CDE4F', 'link': 'https/:www.somelinktomodel.com', 'modelCardHash': b'0x05416460deb76d57af601be17e777b93592d8d4d4a4096c57876a91c84f4a712', 'version': 1}), 'event': 'publishedModel', 'logIndex': 0, 'transactionIndex': 0, 'transactionHash': HexBytes('0x3dfba5ab6910982e9a9198d55949cdbe88cdc103ba8837e400a4adf4634ecae1'), 'address': '0x6Da5800336B730723B2b9242c12b0ADb6467dA6A', 'blockHash': HexBytes('0x931ae5d984e74c5a1eb272aea179dba8435ecc0f46d726d4788246f6a6698172'), 'blockNumber': 76})


As we can see, important properties are saved in the event, it is possible for anyone that has access to the blockchain to check the link, modelcardhash and the version of a certain model by checking the event. The event is for anyone to check. Other events include model certification and updating the model.

In [63]:
#let's see if we can certify the model, let's certify an account
tx_hash = Contract.functions.addCertifier(accounts[1]).transact({'from': accounts[0]})
tx_hash = Contract.functions.certifyModel(test_hash).transact({'from': accounts[1]})
m_version = Contract.functions.getVersion(test_hash).call()
print("the model is certified: " + str(Contract.functions.getCertfication(m_version).call()))

the model is certified: True


In [67]:
#Models can get certified, lets update a model
new_hash = Web3.to_bytes(text = '0x05416460deb76sd7af601be17e777b93592d8d4d4sfdssc57876a91c84f4a712')
new_link = 'https/:www.somelinktonewmodel.com'
tx_hash = Contract.functions.updateModel(new_link, new_hash).transact({'from': accounts[0]})
print(Contract.functions.getLink().call() + ", the link to the model")
m_version = Contract.functions.getVersion(test_hash).call()
print("the " + str(m_version) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version).call()))
m_version_new = Contract.functions.getVersion(new_hash).call()
print("the " + str(m_version_new) + "th version of model is certified: " + str(Contract.functions.getCertfication(m_version_new).call()))
#making sure that the hash from the contract is the same as the input
assert str(Contract.functions.getMostRecentModelCardHash().call()) == str(new_hash)


https/:www.somelinktonewmodel.com, the link to the model
the 1th version of model is certified: True
the 3th version of model is certified: False


In [68]:
endTime1 = datetime.now()
elapsedTime1 = endTime1 - startTime
print("elapsedTime1", elapsedTime1)

elapsedTime1 0:02:00.676639


In [90]:
#check the block of updating a model
event_filter = Contract.events['updatedModel'].create_filter(fromBlock='latest')
new_hash = Web3.to_bytes(text = '0x05416463deb79sd8af2401be17e777b93592d8d4d4sfdssc57876a91c84f4a712')
new_link = 'https/:www.somelinktonewmodelblock.com'

tx_hash = Contract.functions.updateModel(new_link, new_hash).transact({'from': accounts[0]})
block = w3.eth.get_block('latest')
block

AttributeDict({'hash': HexBytes('0x0ae8bef0c610c11674e43a2b73ffbf5a0be8e4f218570f1dbbc28d15ead63c45'),
 'parentHash': HexBytes('0x1c758f41364ce953a9c1f0443783ae47bcc3d5e6fd3bbf5fa10348f38f1ee390'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0x0000000000000000000000000000000000000000',
 'stateRoot': HexBytes('0xa19e566cd11ac3e90d4a18e95011c00242e9a8640ff350b4d8a57b5a4cd1cd24'),
 'transactionsRoot': HexBytes('0x5e08176545350625ab818d60be8ff57169ea9eae07ff4c71b08617ecb84e3b03'),
 'receiptsRoot': HexBytes('0x25667a4590ec11ad7ecebcbbf6d2cdde9031eb5f56d4c488133f19c231b7a928'),
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000200000000000000000000000000000080000200000000000000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000

In [107]:
#lets create a new contract to estimate the gas costs of the functions
ContractGasCheck = w3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = ContractGasCheck.constructor().transact({"from": w3.eth.accounts[0]})
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# We obtain the block number under which it is deployed 
global contract_block
contract_block = w3.eth.block_number
print("The contract is deployed with block number",contract_block,".")
global contract_address
contract_address = tx_receipt.contractAddress
print("The contract has the address", contract_address,".")

ContractGasCheck = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

#original hash/link
test_hash = Web3.to_bytes(text = '0x05416460deb76d57af601be17e777c93592d8d4d4a4096c57876a91c84f4a712')
test_link = 'https/:www.somelinktomodel.com'

#new hash/link for updating model
new_hash = Web3.to_bytes(text = '0x05416463deb79sd8af2401be17e777b94592d8d4d4sfdssc57876a91c84f4a712')
new_link = 'https/:www.somelinktonewmodelblock.com'

#calculating the gast cost in ETH.
gas_add_cert = ContractGasCheck.functions.addCertifier(accounts[1]).estimate_gas({'from':accounts[0]})
gas_del_cert = ContractGasCheck.functions.deleteCertifier(accounts[1]).estimate_gas({'from':accounts[0]})

gas_upload = ContractGasCheck.functions.publishModel(test_link, test_hash).estimate_gas({'from': accounts[0]})
tx_hash = ContractGasCheck.functions.publishModel(test_link, test_hash).transact({'from': accounts[0]})
gas_update = ContractGasCheck.functions.updateModel(new_link, new_hash).estimate_gas({'from': accounts[0]})
gas_certify = ContractGasCheck.functions.certifyModel(test_hash).estimate_gas({'from': accounts[0]})


The contract is deployed with block number 98 .
The contract has the address 0x942d49585f29B130d3E823Ea5081405494b0419C .


In [108]:
#printing gas prices in gwei, eth and euro
print('gas prices in gwei')
print('adding certifier ', gas_add_cert)
print('deleting certifier ', gas_del_cert)
print('uploading model ', gas_upload)
print('updating model ',gas_update )
print('certifying model ', gas_certify)

print('gas prices in ETH')
print('adding certifier ', gas_add_cert* 10e-9)
print('deleting certifier ', gas_del_cert* 10e-9)
print('uploading model ', gas_upload* 10e-9)
print('updating model ',gas_update * 10e-9)
print('certifying model ', gas_certify* 10e-9)

#1 eth is 2055 euro at the moment
print('gas prices in EUR')
print('adding certifier ', gas_add_cert* 10e-9*2056.55)
print('deleting certifier ', gas_del_cert* 10e-9*2056.55)
print('uploading model ', gas_upload* 10e-9*2056.55)
print('updating model ',gas_update * 10e-9*2056.55)
print('certifying model ', gas_certify* 10e-9*2056.55)


gas prices in gwei
adding certifier  46215
deleting certifier  28459
uploading model  291220
updating model  231002
certifying model  56725
gas prices in ETH
adding certifier  0.00046215
deleting certifier  0.00028459
uploading model  0.0029122
updating model  0.00231002
certifying model  0.00056725
gas prices in EUR
adding certifier  0.9504345825
deleting certifier  0.5852735645
uploading model  5.989084910000001
updating model  4.750671631
certifying model  1.1665779875000002


In [101]:
block = w3.eth.get_block(91)
block 

AttributeDict({'hash': HexBytes('0xf179b78f9b96b65650bb05e60056f582773b06ae364637c45b48e0c0798c8140'),
 'parentHash': HexBytes('0x15734bcee66eb91cf229d6384c4af32da3c3cf38d071e9407b05f6e88fe09494'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'miner': '0x0000000000000000000000000000000000000000',
 'stateRoot': HexBytes('0x1f66c67c44daa31f9e9ce2ba1f2b10f0073baae639801389b19fdb274f0fa73c'),
 'transactionsRoot': HexBytes('0xa144c1a71ee4c365a5aebf8ed6b8af077c60e895bf26f3589e021b054fa04635'),
 'receiptsRoot': HexBytes('0x2cc6258c98485f62df10b422f8b8e86d4b3c28ad8252e2b0c2322fca22f5cc3c'),
 'logsBloom': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000